In [1]:
import numpy as np

In [16]:
# def add_dim(x, data = {"depth":0}):
#     if x.shape[0]>0:
#         data["depth"]+=1
#         return np.array([add_dim(x[1:], data) for _ in range(x[0])])
#     else:
#         return Node(0)
        

In [106]:
y = Lattice_2D(x)
type(x)==np.ndarray

True

In [269]:
class Node():
    def __init__(self, lattice, *args):
        self.lattice = lattice
        self.weight   = np.array(args) 
        self.position = np.array(args)
        
    def get_weight(self):
        return self.weight
    
    def get_position(self):
        return self.position
    
    def distance_to(self, vector):
        return self.lattice.distance_function(vector- self.weight)
    
    def __getitem__(self, i):
        return self.position[i]
    
    def __add__(self, other):
        try:
            self.weight = self.weight + other
            return self
        except TypeError:
            raise TypeError(f"type {type(self.weight)} and type {type(other)} can not be + ")
            
    def __neg__(self, other):
        try:
            self.weight = self.weight - other
            return self
        except TypeError:
            raise TypeError(f"type {type(self.weight)} and type {type(other)} can not be - ")
    
    def __mul__(self, other):
        try:
            if 
            self.weight = other*self.weight
            return self
        except TypeError:
            raise TypeError(f"type {type(self.weight)} and type {type(other)} can not be multiplied")
            
            
class Lattice():
    def __init__(self):
        self.lattice = None
        self.distance_function = np.linalg.norm
             
    def __getitem__(self, position):
        return self.lattice[position]   
        
    
    def node_distance(self, node_1, node_2):
        return self.distance_function(node_1.weight-node_2.weight)
    
    def get_node_weight(self, position):
        return self[position].get_weight()
    
        
    def get_weight(self):
        f = np.vectorize(lambda node: node.get_weight(), signature='()->(n)')
        return f(self.lattice)
    
    def get_position(self):
        f = np.vectorize(lambda node: node.get_position(), signature='()->(n)')
        return f(self.lattice)
    
    
    
    def relative_distance(self, ref_vector):
        f = np.vectorize(lambda node: node.distance_to(ref_vector))
        return f(self.lattice)
    
    def find_closest_id(self, ref_vector):
        distance_matrix = self.relative_distance(ref_vector)
        return np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)
    
    def find_closest(self, ref_vector):
        return self.lattice[self.find_closest_id(ref_vector)]
    
#     epsilon, sigma, h
#     def h(i, j, t):
#         return exp(-d(i, j)**2/(2*sigma(t)**2))
    
#     def sigma(t):
#         sigma_0*exp(-t/tmax)
        
        

        
    
    
    def train(self, input):
        
        class DecreasingExp():
            def __init__(self, x_0, x_max):
                self.x_0 = x_0
                self.x_max = x_max
            def value(self, x):
                return self.x_0*np.exp(-x/self.x_max)
        
        
        class Normal():
            def __init__(self, node_distance, epsilon, sigma):
                self.distance = node_distance
                self.epsilon  = epsilon
                self.sigma    = sigma
            def value(self, node_i, node_j, t):
                epsilon_t = self.epsilon.value(t)
                d_ij      = self.distance(node_i, node_j)
                sigma_t   = self.sigma.value(t)
                return epsilon_t*np.exp(-d_ij**2/(2*sigma_t**2))

        class Update():
            def __init__(self, node_distance, 
                         epsilon_0 = 1, epsilon_max = 100, 
                         sigma_0   = 1, sigma_max   = 100):
                self.node_distance = node_distance
                self.epsilon  = DecreasingExp(epsilon_0, epsilon_max)
                self.sigma    = DecreasingExp(sigma_0, sigma_max)
                self.coef     = Normal(self.node_distance, self.epsilon, self.sigma)

            def value(self, reference_vector, best_matching_node, current_node, t):
                return self.coef.value(best_matching_node, current_node, t)
        
        
        
        best_matching_node = self.find_closest(input)
        update = Update(self.node_distance)
        
        for node in self:
            print(update.value(reference_vector = input, 
                   best_matching_node = best_matching_node,
                   current_node = node,
                   t = 0) )
    # TODO : add an iterable here, to avoid thinking about i, j (, k, ...)
    
    
    def __iter__(self):
        pass
    
    
    
    
    
class Lattice_1D(Lattice):
    def __init__(self, list_dim):
        super().__init__()
        self.lattice = np.array([Node(self, i) for i in range(list_dim[0])])
                
    def __iter__(self):
        for i in xrange(self.list_dim[0]):
            yield self.lattice[i]

            
class Lattice_2D(Lattice):
    def __init__(self, list_dim):
        super().__init__()
        self.list_dim = list_dim
        self.lattice = np.array([[Node(self, i, j) for j in range(list_dim[1])] 
                                                   for i in range(list_dim[0])])
        
    
    def __iter__(self):
        for i in range(self.list_dim[0]):
            for j in range(self.list_dim[1]):
                yield self.lattice[i][j]

In [270]:
x = np.array([3, 5])

In [271]:
y = Lattice_2D(x)

In [272]:
np.array(y.lattice).shape

(3, 5)

In [273]:
w = y.get_weight()

In [274]:
a = y[1, 0]
b = y[1, 3]

In [275]:
y.node_distance(a, b)

3.0

In [276]:
y[1, 3].weight

array([1, 3])

In [277]:
y.find_closest(y[1,0].weight+np.array([1, 3]))

In [278]:
y.find_closest_id(y[1,0].weight+np.array([1, 3]))

(2, 3)

In [279]:
y.lattice[y.find_closest_id(y[1,0].weight+np.array([1, 3]))]

In [280]:
y.train(np.array([0.75, 0.75]))

0.3678794411714422
0.6065306597126334
0.3678794411714422
0.08208499862389876
0.006737946999085461
0.6065306597126334
1.0
0.6065306597126334
0.1353352832366127
0.011108996538242306
0.3678794411714422
0.6065306597126334
0.3678794411714422
0.08208499862389876
0.006737946999085461


In [281]:
y[1,0].position

array([1, 0])

In [282]:
y[1,1].weight+np.array([1,1.6])

array([2. , 2.6])

In [283]:
(y[1,1]+np.array([1,2.3])).weight

array([2. , 3.3])

In [284]:
(y[1,1]*3).weight

array([6. , 9.9])

In [285]:
(y[1,1]*0.1).weight

array([0.6 , 0.99])

In [288]:
(y[1,1]*y[1,1])

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
x = np.array([3])

In [ ]:
y = Lattice_1D(x)

In [14]:
y.get_weight()

array([[[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4]],

       [[1, 0],
        [1, 1],
        [1, 2],
        [1, 3],
        [1, 4]],

       [[2, 0],
        [2, 1],
        [2, 2],
        [2, 3],
        [2, 4]]])

In [15]:
y.train(np.array([0.75]))

[[1.41421356 1.         1.41421356 2.23606798 3.16227766]
 [1.         0.         1.         2.         3.        ]
 [1.41421356 1.         1.41421356 2.23606798 3.16227766]]


AttributeError: 'Node' object has no attribute 'update'

In [16]:
np.array([[1, 1],[1,2]])[(1,1)]

2

In [17]:
np.array([1, 1, 5])[(2,)]

5